# Aggregates in Iceberg

In [17]:
from pyspark.sql import SparkSession
import os

In [18]:
spark = (
    SparkSession.builder
    .appName("Aggregates in Iceberg")
    .master("spark://spark:7077") 
    .getOrCreate()
)

Full Aggregation Report, `count`, `sum`, `round`. Joining all the other data tables (which are likely from varying sources)

In [19]:
spark.sql("""
SELECT
    COUNT(DISTINCT o.id) AS total_orders,
    COUNT(DISTINCT c.id) AS total_customers,
    SUM(oi.quantity) AS total_items_sold,
    ROUND(SUM(oi.quantity * p.price), 2) AS total_revenue
FROM ice.demo.orders o
JOIN ice.demo.order_items oi ON o.id = oi.order_id
JOIN ice.demo.products p ON oi.product_id = p.product_id
JOIN ice.demo.customers c ON o.customer_id = c.id
""").show()

+------------+---------------+----------------+-------------+
|total_orders|total_customers|total_items_sold|total_revenue|
+------------+---------------+----------------+-------------+
|          19|             19|             216|     16737.84|
+------------+---------------+----------------+-------------+



Here, let's use `CASE` and `WHEN` which is available for our query engine, Spark. Every query engine has different powerful querying possibilities

In [20]:
spark.sql("""
SELECT
    CASE
        WHEN p.price < 50 THEN 'Low-cost'
        WHEN p.price BETWEEN 50 AND 150 THEN 'Mid-range'
        ELSE 'Premium'
    END AS price_tier,
    COUNT(DISTINCT oi.id) AS items_sold,
    ROUND(SUM(oi.quantity * p.price), 2) AS total_revenue
FROM ice.demo.order_items oi
JOIN ice.demo.products p ON oi.product_id = p.product_id
GROUP BY price_tier
ORDER BY total_revenue DESC
""").show()

+----------+----------+-------------+
|price_tier|items_sold|total_revenue|
+----------+----------+-------------+
| Mid-range|        15|      3739.58|
|   Premium|         4|      2699.92|
|  Low-cost|        21|      2389.36|
+----------+----------+-------------+



## `ROLLUP`

Let's perform a `ROLLUP` which provides layers of analysis:

1. Totals per (customer, product),
2. Totals per customer,
3. Grand total


In [21]:
spark.sql("""
SELECT
    customer_name,
    product_name,
    ROUND(SUM(base.quantity * base.price), 2) AS total_spent
FROM (
    SELECT
        c.name  AS customer_name,
        p.name  AS product_name,
        oi.quantity,
        p.price
    FROM ice.demo.orders o
    JOIN ice.demo.customers c ON o.customer_id = c.id
    JOIN ice.demo.order_items oi ON o.id = oi.order_id
    JOIN ice.demo.products p ON oi.product_id = p.product_id
) AS base
GROUP BY ROLLUP (customer_name, product_name)
ORDER BY customer_name NULLS LAST, total_spent DESC
""").show()

+-------------+-------------------+-----------+
|customer_name|       product_name|total_spent|
+-------------+-------------------+-----------+
|  Alice Smith|               NULL|    1679.84|
|  Alice Smith|     Gaming Monitor|    1199.96|
|  Alice Smith|          USB-C Hub|     479.88|
| Aria Johnson|               NULL|     739.92|
| Aria Johnson|         Smartwatch|     599.96|
| Aria Johnson|   Book: Clean Code|     139.96|
| Ava Thompson|               NULL|     879.88|
| Ava Thompson|Mechanical Keyboard|     639.92|
| Ava Thompson|       Coffee Maker|     239.96|
| Benjamin Kim|               NULL|    1739.92|
| Benjamin Kim|      Standing Desk|    1599.96|
| Benjamin Kim|       Laptop Stand|     139.96|
|  Bob Johnson|               NULL|     399.88|
|  Bob Johnson|       Laptop Stand|     279.92|
|  Bob Johnson|   Wireless Charger|     119.96|
|  Carol Adams|               NULL|     519.88|
|  Carol Adams|          USB-C Hub|     319.92|
|  Carol Adams|          Webcam HD|     

## `HAVING`

`HAVING` is for queries that have a `GROUP BY` where additional querying needs to be performed after the grouping

In [22]:
spark.sql("""
SELECT
    c.name AS customer_name,
    ROUND(AVG(order_total), 2) AS avg_order_value
FROM (
    SELECT
        o.id AS order_id,
        o.customer_id,
        SUM(oi.quantity * p.price) AS order_total
    FROM ice.demo.orders o
    JOIN ice.demo.order_items oi ON o.id = oi.order_id
    JOIN ice.demo.products p ON oi.product_id = p.product_id
    GROUP BY o.id, o.customer_id
) joined_orders
JOIN ice.demo.customers c ON joined_orders.customer_id = c.id
GROUP BY c.name
HAVING AVG(order_total) > 200
ORDER BY avg_order_value DESC
""").show()

+--------------+---------------+
| customer_name|avg_order_value|
+--------------+---------------+
|   Jack Wilson|        1779.92|
|  Benjamin Kim|        1739.92|
|   Alice Smith|        1679.84|
|  Lucas Martin|        1399.88|
|Isabella Rossi|        1119.88|
| Noah Williams|         959.88|
|  Ava Thompson|         879.88|
| Liam O’Connor|         839.88|
| Sofia Almeida|         819.84|
| Emma Robinson|         799.84|
|  Aria Johnson|         739.92|
|    Maya Patel|         719.88|
| Diego Ramirez|         599.92|
|      Zoe King|         579.88|
|   Carol Adams|         519.88|
| Olivia Garcia|         499.88|
|   Bob Johnson|         399.88|
|  James Nguyen|         339.92|
|    Ethan Chen|         319.92|
+--------------+---------------+



## `CUBE`

`CUBE` computes all combinations of the grouping dimensions. Very useful for BI rollups and time-based insights.

In [36]:
spark.sql("""
SELECT
    week,
    customer_name,
    price_tier,
    ROUND(SUM(quantity * price), 2) AS total_spent
FROM (
    SELECT
        date_trunc('week', CAST(o.timestamp AS DATE)) AS week,
        c.name AS customer_name,
        CASE
            WHEN p.price < 50 THEN 'Low-cost'
            WHEN p.price BETWEEN 50 AND 150 THEN 'Mid-range'
            ELSE 'Premium'
        END AS price_tier,
        oi.quantity,
        p.price
    FROM ice.demo.orders o
    JOIN ice.demo.customers c ON o.customer_id = c.id
    JOIN ice.demo.order_items oi ON o.id = oi.order_id
    JOIN ice.demo.products p ON oi.product_id = p.product_id
) base
GROUP BY CUBE (week, customer_name, price_tier)
ORDER BY week, customer_name
""").show(200, truncate=False)

+-------------------+--------------+----------+-----------+
|week               |customer_name |price_tier|total_spent|
+-------------------+--------------+----------+-----------+
|NULL               |NULL          |NULL      |16737.84   |
|NULL               |NULL          |Low-cost  |4378.8     |
|NULL               |NULL          |Mid-range |6959.2     |
|NULL               |NULL          |Premium   |5399.84    |
|NULL               |Alice Smith   |Premium   |1199.96    |
|NULL               |Alice Smith   |NULL      |1679.84    |
|NULL               |Alice Smith   |Low-cost  |479.88     |
|NULL               |Aria Johnson  |Low-cost  |139.96     |
|NULL               |Aria Johnson  |Mid-range |599.96     |
|NULL               |Aria Johnson  |NULL      |739.92     |
|NULL               |Ava Thompson  |NULL      |879.88     |
|NULL               |Ava Thompson  |Mid-range |879.88     |
|NULL               |Benjamin Kim  |Premium   |1599.96    |
|NULL               |Benjamin Kim  |NULL

In [37]:
spark.stop()